# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(len(file_path_list))

/Users/xuhao3/UDACITY/Data_engineering_ND/Project2-Apache-Cassandra
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
column_name = ["artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"]    
# for every filepath in the file path list 
for f in file_path_list:
    df = pd.read_csv(f)
    value = df[column_name].values
    for row in value:
        full_data_rows_list.append(row)
    
small_df = pd.DataFrame(data = full_data_rows_list, columns = column_name)
small_df.to_csv('event_datafile_new1.csv')


In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)


8056
[['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26'], ['Train', 'Logged In', 'Ryan', 'M', '2', 'Smith', '205.45261', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Marry Me', '200', '1.54224E+12', '26'], ['', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '', 'free', 'Eureka-Arcata-Fortuna, CA', 'GET', 'Home', '1.54087E+12', '563', '', '200', '1.54225E+12', '9'], ['', 'Logged In', 'Austin', 'M', '0', 'Rosales', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54106E+12', '521', '', '200', '1.54225E+12', '12'], ['Sony Wonder', 'Logged In', 'Samuel', 'M', '0', 'Gonzalez', '218.06975'

In [5]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [30]:
# check the number of rows in your csv file
with open('event_datafile_new1.csv', 'r', encoding = 'utf8') as f:
#    print(sum(1 for line in f))
    for line in f:
        print(line)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId

0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26.0

1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26.0

2,Train,Ryan,M,2,Smith,205.45261000000002,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26.0

3,,Wyatt,M,0,Scott,,free,"Eureka-Arcata-Fortuna, CA",563,,9.0

4,,Austin,M,0,Rosales,,free,"New York-Newark-Jersey City, NY-NJ-PA",521,,12.0

5,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61.0

6,,Samuel,M,1,Gonzalez,,free,"Houston-The Woodlands-Sugar Land, TX",597,,61.0

7,,,,0,,,paid,,602,,

8,,Tegan,F,1,Levine,,paid,"Portland-South Portland, ME",602,,80.0

9,Van Halen,Tegan,F,2,Levine,289.38403999999997,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80.0

10,Magic Sam,Tegan,F,3,Levine,132.048

1945,Black Eyed Peas,Chloe,F,9,Cuevas,229.61587000000003,paid,"San Francisco-Oakland-Hayward, CA",758,Let's Get It Started,49.0

1946,OneRepublic,Ayla,F,3,Johnson,208.14321999999999,free,"Santa Rosa, CA",729,Apologize,63.0

1947,Charlie Louvin,Chloe,F,10,Cuevas,170.86649,paid,"San Francisco-Oakland-Hayward, CA",758,I Think I'll Live,49.0

1948,Ill Nino,Chloe,F,11,Cuevas,202.57914,paid,"San Francisco-Oakland-Hayward, CA",758,Unframed (Album Version),49.0

1949,Coldplay,Chloe,F,12,Cuevas,294.1122,paid,"San Francisco-Oakland-Hayward, CA",758,Fix You,49.0

1950,Kansas,Chloe,F,13,Cuevas,275.9571,paid,"San Francisco-Oakland-Hayward, CA",758,Carry On Wayward Son,49.0

1951,,Lily,F,0,Burns,,free,"New York-Newark-Jersey City, NY-NJ-PA",780,,32.0

1952,Kurtis Blow,Colm,M,0,Santana,353.88036,free,"Nashville-Davidson--Murfreesboro--Franklin, TN",722,The Breaks,67.0

1953,Chromeo,Sylvie,F,0,Cruz,171.57179,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",717,Momma's Boy,10.0

1954,Beach House,Chl


3445,,Tegan,F,3,Levine,,paid,"Portland-South Portland, ME",933,,80.0

3446,,Tegan,F,4,Levine,,paid,"Portland-South Portland, ME",933,,80.0

3447,The Rolling Stones,Ryann,F,2,Smith,271.49016,free,"Palestine, TX",938,Angie (1993 Digital Remaster),92.0

3448,,Tegan,F,5,Levine,,paid,"Portland-South Portland, ME",933,,80.0

3449,Enrique Iglesias,Ryann,F,3,Smith,264.202,free,"Palestine, TX",938,Hero,92.0

3450,Deadmau5,Tegan,F,6,Levine,429.94892999999996,paid,"Portland-South Portland, ME",933,Arguru,80.0

3451,Aphex Twin,Ryann,F,4,Smith,547.1342400000001,free,"Palestine, TX",938,Tha,92.0

3452,Metric,Tegan,F,7,Levine,286.24934,paid,"Portland-South Portland, ME",933,Help I'm Alive,80.0

3453,Warren Barfield,Tegan,F,8,Levine,222.98077,paid,"Portland-South Portland, ME",933,Love Is Not A Fight,80.0

3454,Jack Johnson,Ryann,F,5,Smith,283.92444,free,"Palestine, TX",938,They Do_ They Don't,92.0

3455,Akon,Tegan,F,9,Levine,235.85914,paid,"Portland-South Portland, ME",933,Lonely,80.0

3456,Pixies,R

5374,Lil Wayne / Eminem,Lily,F,60,Koch,229.58975,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,Drop The World,15.0

5375,Damage,Lily,F,61,Koch,263.18322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,For Your Pleasure,15.0

5376,Rionegro / SolimÃÂµes,Lily,F,62,Koch,280.92036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,De SÃÂ£o Paulo ÃÂ BelÃÂ©m,15.0

5377,Drive-By Truckers,Lily,F,63,Koch,246.22975,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,Outfit,15.0

5378,Tweet,Lily,F,64,Koch,256.39137999999997,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,Smoking Cigarettes (LP Version),15.0

5379,Ben Folds Five,Lily,F,65,Koch,207.882,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,Steven's Last Night In Town,15.0

5380,Glen Hansard,Lily,F,66,Koch,239.59465,paid,"Chicago-Naperville-Elgin, IL-IN-WI",172,Lies,15.0

5381,Chevelle,Adler,M,0,Barrera,389.85098,free,"New York-Newark-Jersey City, NY-NJ-PA",174,Dos (LP Version),100.0

5382,Bill Withers,Lily,F,67,Koch,383.73832000000004,paid,"Chicago-Na


6944,Sublime,Kate,F,5,Harrell,172.5122,paid,"Lansing-East Lansing, MI",817,Right Back,97.0

6945,Andres Calamaro,Kate,F,6,Harrell,276.13995,paid,"Lansing-East Lansing, MI",817,Flaca,97.0

6946,Avril Lavigne,Chloe,F,9,Cuevas,240.48281,paid,"San Francisco-Oakland-Hayward, CA",816,When You're Gone,49.0

6947,Metronomy,Chloe,F,10,Cuevas,315.24526000000003,paid,"San Francisco-Oakland-Hayward, CA",816,Heartbreaker [Discodeine Remix],49.0

6948,Andres Calamaro,Kate,F,7,Harrell,278.12526,paid,"Lansing-East Lansing, MI",817,El Cantante,97.0

6949,Good Clean Fun,Kate,F,8,Harrell,116.61016000000001,paid,"Lansing-East Lansing, MI",817,Its Fun To Be A Vampire&,97.0

6950,Zero 7,Chloe,F,11,Cuevas,259.05587,paid,"San Francisco-Oakland-Hayward, CA",816,Mr McGee,49.0

6951,The Stranglers,Kate,F,9,Harrell,211.90485,paid,"Lansing-East Lansing, MI",817,Golden Brown,97.0

6952,Snow Patrol,Chloe,F,12,Cuevas,197.09342,paid,"San Francisco-Oakland-Hayward, CA",816,Shut Your Eyes,49.0

6953,Mondo Marcio,Kate,

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [1]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "<ENTER INSERT STATEMENT HERE>"
        query = query + "<ASSIGN VALUES HERE>"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[#], line[#]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


                    

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()